<a href="https://colab.research.google.com/github/IfeakanduBenedict/pediatric-pneumonia-ml-england/blob/main/Pediatric_Pneumonia_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Topic: Predicting Pediatric Pneumonia Hospitalisation Rates in English Local Authorities**
   
## Research Questions

**PRIMARY**: To what extent can machine learning models predict emergency pneumonia hospitalisation rates among children and young people under 19 across English Local Authorities using socioeconomic deprivation, demographic vulnerability, and healthcare utilisation indicators?

**SECONDARY**: Which socioeconomic, demographic, and healthcare-related factors contribute most to geographic inequalities in pneumonia hospitalisation rates among children and young people under 19 across England?


# **Notebook Setup and Installation of Needed Library**